In [2]:
import os
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision
from torchvision.transforms.transforms import ToTensor

from models.model import My_siamese
from utils.utils import increment_path
from pathlib import Path

from tqdm import tqdm
import wandb

import numpy as np
import matplotlib.pyplot as plt

from dataset.reconstruct import prepare_data
from dataset.loader import train_validation_loader
from dataset.loader import testset_loader

In [3]:

net = My_siamese().cuda()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0005)

data_dir = os.path.join("data")
datasets.Omniglot(data_dir, background=True, download=True, transform=ToTensor())
datasets.Omniglot(data_dir, background=False, download=True, transform=ToTensor())


Files already downloaded and verified
Files already downloaded and verified


Dataset Omniglot
    Number of datapoints: 0
    Root location: data/omniglot-py
    StandardTransform
Transform: ToTensor()

In [4]:
train_dir, val_dir, test_dir = prepare_data(data_dir, 3)
train_loader, val_loader = train_validation_loader(train_dir, val_dir, 8, False, 5, True, 3, 2)
   

In [11]:
net.train()
train_loss = 0
sig = nn.Sigmoid()

for i, (img1, img2, label) in tqdm(enumerate(train_loader), total=len(train_loader)):

    output = net(img1.cuda(), img2.cuda())
    loss = loss_fn(output, label.cuda())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss

    if (i+1) % (len(train_loader)//5) == 0 or i == (len(train_loader) -1):
        # wandb.log({"Train_loss":train_loss/i})
        print("Train_loss", train_loss/i)

train_loss /= len(train_loader)
print("Train_loss", train_loss)


 20%|██        | 490/2445 [00:46<03:05, 10.54it/s]

Train_loss tensor(0.4583, device='cuda:0', grad_fn=<DivBackward0>)


 40%|████      | 979/2445 [01:34<02:20, 10.46it/s]

Train_loss tensor(0.4336, device='cuda:0', grad_fn=<DivBackward0>)


 60%|██████    | 1468/2445 [02:21<01:36, 10.10it/s]

Train_loss tensor(0.4154, device='cuda:0', grad_fn=<DivBackward0>)


 80%|████████  | 1958/2445 [03:07<00:46, 10.54it/s]

Train_loss tensor(0.3994, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████| 2445/2445 [03:55<00:00, 10.38it/s]

Train_loss tensor(0.3832, device='cuda:0', grad_fn=<DivBackward0>)
Train_loss tensor(0.3830, device='cuda:0', grad_fn=<DivBackward0>)


In [5]:
load = torch.load("./test.pt")
net.load_state_dict(load["model"])

<All keys matched successfully>

In [16]:
net.eval()
val_loss = 0

from PIL import Image
import cv2

with torch.no_grad():    
    for i, (img1, img2, label) in tqdm(enumerate(val_loader), total=len(train_loader)):

        output = net(img1.cuda(), img2.cuda())
        loss = loss_fn(output, label.cuda())

        val_loss += loss

        if (i+1) % (len(val_loader)//5) == 0 or i == (len(val_loader) -1):
            # wandb.log({"Train_loss":train_loss/i})
            print("Val loss", val_loss/i)

        if i==0:
            
            show_img = torch.cat((img1[0].unsqueeze(0),img2[:5]))
            print("show img : ", show_img.shape)
            similarity = output
            print("similarity : ", similarity)
            print("label : ", label)

            f_img = torch_to_cv2(show_img)
            cv2.imwrite("./test2.jpg", f_img)
            s_img = put_result(f_img, similarity, label, 5)
            cv2.imwrite("./test3.jpg", s_img)

            break

    val_loss /= len(val_loader)
    print("Val", val_loss)




  0%|          | 0/2445 [00:00<?, ?it/s]

show img :  torch.Size([6, 1, 105, 105])
similarity :  tensor([[ 1.5764],
        [ 0.6041],
        [ 1.0594],
        [-1.2201],
        [ 1.7759]], device='cuda:0')
label :  tensor([[1.],
        [0.],
        [0.],
        [0.],
        [0.]])
Val tensor(0.0007, device='cuda:0')


In [37]:
ckpt = {'model':net.state_dict(),
                    'optimizer':optimizer.state_dict(),}
torch.save(ckpt, "./test.pt")

In [7]:
def torch_to_cv2(imgs):

    for i, img in enumerate(imgs):

        img = img.cpu()
        img = img.numpy().transpose(1,2,0)
        img = (img -img.min())/(img.max()-img.min())*255

        if i ==0:
            cv_img = img
        else : 
            cv_img = np.concatenate((cv_img, img), axis=1)

    return cv_img

In [12]:
def put_result(img, similar, label, candi):

    split = candi + 1

    w = img.shape[1]//split

    for i in range(split):
        
        if i ==0:
            cv2.putText(img, "Template", (w//4, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, 0,2)
        else : 
            s = similar[i-1].item()
            l = int(label[i-1].item())
            cv2.putText(img, f"Result : {s:.2f}", (w//9 + w*i, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.4, 0,2)
            cv2.putText(img, f"Label : {int(l)}", (w//4 + w*i, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, 0,2)

    return img


In [17]:
test_loader = testset_loader(test_dir, 5, 3, 2)
    


with torch.no_grad():    
    for i, (img1, img2, label) in tqdm(enumerate(test_loader), total=len(train_loader)):

        output = net(img1.cuda(), img2.cuda())
        loss = loss_fn(output, label.cuda())

        val_loss += loss

        if (i+1) % (len(test_loader)//5) == 0 or i == (len(test_loader) -1):
            # wandb.log({"Train_loss":train_loss/i})
            print("Val loss", val_loss/i)

        if i==0:
            
            show_img = torch.cat((img1[0].unsqueeze(0),img2[:5]))
            print("show img : ", show_img.shape)
            similarity = output
            print("similarity : ", similarity)
            print("label : ", label)

            f_img = torch_to_cv2(show_img)
            cv2.imwrite("./test_2.jpg", f_img)
            s_img = put_result(f_img, similarity, label, 5)
            cv2.imwrite("./test_3.jpg", s_img)

            break

    val_loss /= len(test_loader)
    print("Val", val_loss)




  0%|          | 0/2445 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f4450cc59d0>
Traceback (most recent call last):
Exception ignored in:   File "/home/jwk6844/anaconda3/envs/track/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f4450cc59d0>    
self._shutdown_workers()Traceback (most recent call last):

  File "/home/jwk6844/anaconda3/envs/track/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
  File "/home/jwk6844/anaconda3/envs/track/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
        if w.is_alive():self._shutdown_workers()

  File "/home/jwk6844/anaconda3/envs/track/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
  File "/home/jwk6844/anaconda3/envs/track/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers

show img :  torch.Size([6, 1, 105, 105])
similarity :  tensor([[ 1.9571],
        [-3.1557],
        [ 0.1665],
        [-2.1002],
        [-1.1059]], device='cuda:0')
label :  tensor([[1.],
        [0.],
        [0.],
        [0.],
        [0.]])
Val tensor(0.0002, device='cuda:0')
